### Import librairies

In [2]:
import torch
import pandas as pd
import os,glob
import numpy as np
import time
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns 
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import stats
import torch,random
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader, TensorDataset, Dataset
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import explained_variance_score
from scipy import interpolate

import warnings
warnings.filterwarnings("ignore")

seed = 42
##### setting deterministic setting for torch
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")
plt.rcParams["figure.figsize"] = (10,3)



### read data

In [6]:
# g-t files
path1 = './training_set/'
path2 = './development_set/'

file1 = 'train_scores.csv'
file2 = 'dev_scores.csv'

features_path1 = 'C3D/'
features_path2 = './Features/efficientnet_b3/'
features_path3 = './Features/densenet121/'

# get data
df1 = pd.read_csv(path1+file1,sep=',')
df2 = pd.read_csv(path2+file2,sep=',')

data1 = df1[["video_id","scores_short_term","decay_alpha"]]
data2 = df2[["video_id","scores_short_term","decay_alpha"]]


### Read C3D features

In [7]:
# read C3D features
def read_csv_feat(file):
    df = pd.read_csv(file,sep=',')
    feat = df.columns.values
    #feat = [f1.strip(" ") for f1 in feat]
    feat = [float('0.'+f1.split(".")[-1]) if len(f1.split("."))>2 else float(f1) for f1 in feat]

    return feat

In [8]:
X_train,X_test = [],[]
y_train,y_test = [],[]

lst = os.listdir(path1+features_path1)
lst2 = os.listdir(path2+features_path1)

start = time.time()
for i in range(len(lst)):
    f = lst[i]
    label = data1.loc[data1['video_id'] == int(f.split('.')[0])]['scores_short_term']
    feat = read_csv_feat(path1+features_path1+f)
    X_train.append(feat)
    y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)

print('training data processed: ',X_train.shape,y_train.shape)
print('[INFO]',time.time() - start)
start = time.time()

for i in range(len(lst2)):
    f = lst2[i]
    label = data2.loc[data2['video_id'] == int(f.split('.')[0])]['scores_short_term']
    feat = read_csv_feat(path2+features_path1+f)
    X_test.append(feat)
    y_test.append(label)

    
X_test = np.array(X_test)
y_test = np.array(y_test)

print('val data processed: ',X_test.shape, y_test.shape)
print('[INFO]',time.time() - start)

training data processed:  (7000, 4096) (7000, 1)
[INFO] 3110.6872770786285
val data processed:  (1500, 4096) (1500, 1)
[INFO] 610.2185342311859


### Read Resnet features

In [ ]:
#training data
X_train = []
y_train = []
lst = sorted(os.listdir(path1+features_path1))

start = time.time()
for f1, f2, f3 in zip(*[iter(lst)]*3):
    label = data1.loc[data1['video_id'] == int(f1.split('-')[0])]['scores_short_term'].values
    feat = [np.load(path1+features_path1+f1),np.load(path1+features_path1+f2),np.load(path1+features_path1+f3)]
    X_train.append(np.array(feat).squeeze())
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
print('data processed: ',X_train.shape,y_train.shape)
print('[INFO]',time.time() - start)

## validation data
X_test = []
y_test = []
lst2 = sorted(os.listdir(path2+features_path1))

start = time.time()
for f1, f2, f3 in zip(*[iter(lst2)]*3):
    label = data2.loc[data2['video_id'] == int(f1.split('-')[0])]['scores_short_term'].values
    feat = [np.load(path2+features_path1+f1),np.load(path2+features_path1+f2),np.load(path2+features_path1+f3)]
    X_test.append(np.array(feat).squeeze())
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)
print('data processed: ',X_test.shape,y_test.shape)
print('[INFO]',time.time() - start)

### Quick test ==> C3D features with SVM 

In [ ]:
regr = SVR(kernel='rbf',C=100,gamma=1e-3)
#regr = MultiOutputRegressor(model)
SVM_reg = regr.fit(X_train,y_train)
svm_preds = SVM_reg.predict(X_test)

# performance
print('-------------------')
print('|Model performance|')
print('-------------------')

corr1 = float(spearmanr(svm_preds,y_test)[0])
corr2 = float(pearsonr(svm_preds,y_test)[0])
mse = mean_squared_error(svm_preds,y_test)
r_square = r2_score(svm_preds,y_test)
res = explained_variance_score(svm_preds,y_test)


print(f"MSE: {round(mse,4)}")
print(f"R_square: {round(r_square,4)}")
print(f"score: {round(res,4)}")

print(f"Spearman corr: {round(corr1,4)}")
print(f"Pearson corr: {round(corr2,4)}")

### Divide labels into high and low scores

In [ ]:

th1,th2 = 0.7,0.9
th = .8
idx11 = np.argwhere(y_train>=th)[:,0]
idx22 = np.argwhere(y_train<th)[:,0]
idx33,_ = np.where((y_train>th1) & (y_train<th2))

y_train11 = np.array([y_train[i] if i in idx11 else 0 for i in range(len(y_train))])
y_train22 = np.array([y_train[i] if i in idx22 else 0 for i in range(len(y_train))])

y_train1,y_train2= np.expand_dims(y_train11,axis=1),np.expand_dims(y_train22,axis=1)
labels = np.concatenate((y_train1,y_train2),axis=1).astype(float)


### Training with Neural network

### 1. Linear model

In [133]:
class non_LinearNet(nn.Module):
    def __init__(self):
        super(non_LinearNet, self).__init__()
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(1024,512)
        self.fc4 = nn.Linear(512,128)
        self.fc5 = nn.Linear(512,2)

        self.act2 = nn.Tanh()
        self.act =  nn.Sigmoid()
        self.drop = nn.Dropout(p=0.0)
        self.sigmoid = nn.Sigmoid()
        self.bn = nn.BatchNorm1d(2)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(1)

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        #x = self.act(self.fc3(x))
        #x = self.act(self.fc4(x))
        out = self.fc5(x)
        return out

### 2. CNN Model

In [134]:
class convNet(nn.Module):
     def __init__(self): 
        super(convNet,self).__init__()
        self.layer1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1) 
        self.act1 = nn.Sigmoid()
        self.act2 = nn.AdaptiveMaxPool1d(output_size=1024) 
        
        self.layer2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=1) 
        self.act3 = nn.Tanh() 
        self.act4 = nn.AdaptiveMaxPool1d(output_size=512)
        
        self.layer3 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=1) 
        self.act5 = nn.Tanh() 
        self.act6 = nn.AdaptiveMaxPool1d(output_size=128)
        
        self.layer4 = nn.Conv1d(in_channels=64, out_channels=1, kernel_size=1) 
        self.act7 = nn.Tanh() 
        self.act8 = nn.AdaptiveMaxPool1d(output_size=64)
        self.linear_layer = nn.Linear(in_features=64, out_features=2)
    
     def forward(self, x): 
            
        x = self.act3(self.act2(self.layer1(x))) 
        x = self.act3(self.act4(self.layer2(x))) 
        x = self.act3(self.act6(self.layer3(x))) 
        x = self.act3(self.act8(self.layer4(x))) 
        x = x.reshape(x.shape[0], -1) 
        out = self.linear_layer(x) 
        return out



### 3. Vision Transformer model

In [191]:
import cv2
from vit_pytorch import ViT

IM1 = X_train.reshape(X_train.shape[0],64,64)
IM2 = X_test.reshape(X_test.shape[0],64,64)

imgs = np.array([cv2.merge((imgray,imgray,imgray)) for imgray in IM1])
imgs_te = np.array([cv2.merge((imgray,imgray,imgray)) for imgray in IM2])


vit_model = ViT(
    image_size = 64,
    patch_size = 8,
    num_classes = 2,
    dim = 16,
    depth = 8,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.0,
    emb_dropout = 0.0
)

### 4. LSTM model

In [ ]:
class LSTM_model(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2,num_layers, output_size):
        super().__init__()
        self.hidden = None
        self.hidden_size = hidden_size
        self.hidden_size2 = hidden_size2
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size2)
        self.fc2 = nn.Linear(hidden_size2, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
     
        #out, self.hidden = self.lstm(x, self.hidden)  
        out, (final_hidden_state, final_cell_state) = self.lstm(x, self.hidden)  
        #out = out.reshape(-1, self.hidden_size)       
        #out = self.fc(out)  
        out = self.sig(self.fc1(final_hidden_state[-1]))
        out = self.fc2(out)
        
        return out 

### init parameters

In [196]:
bsize = 64
lr = 0.0001
epochs = 50

tr_dataset = TensorDataset(torch.tensor(imgs),torch.tensor(labels)) # use y_train instead of labels for one label
valid_dataset = TensorDataset(torch.tensor(imgs_te),torch.tensor(y_test)) 

train_loader = torch.utils.data.DataLoader(dataset=tr_dataset,batch_size=bsize,shuffle=True,num_workers=0)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,batch_size=bsize,shuffle=True,num_workers=0)

#model = convNet()
#model = non_LinearNet()
#model = vit_model()

model = LSTM_model(2048, 512,128, 3, 2)
params = list(model.parameters())

optimizer = optim.Adam(params, lr=lr)    
#scheduler = MultiStepLR(optimizer, milestones=[100,200], gamma=0.5)
criterion = nn.MSELoss()

### start training 

In [ ]:
start = time.time()
tr_loss = []
te_loss = []

for epoch in tqdm(range(epochs)):  
    train_loss, test_loss, pcc = 0.0, 0.0, 0.0,
        for data1 in train_loader:

        inputs1,labels1,y = data1
        # if model != lstm
        #inputs1 = inputs1.unsqueeze(1).float()
        labels1 = labels1.squeeze().float()
        y = y.squeeze().float()
        
        optimizer.zero_grad()
        #if model == vision transformer
        #outputs1= v(inputs1.permute(0,3,1,2).float())
        outputs1  = outputs1.squeeze()
        loss1 = criterion(outputs1, labels1)#+ criterion(torch.sum(outputs1,1),y)
        train_loss += loss1.item()
        loss1.backward()
        optimizer.step()
            
    #scheduler.step()

    print('[INFO]: start testing')
    
    with torch.no_grad():
        for data2 in tqdm(valid_loader):        
            inputs2, labels2 = data2
            #inputs2 =  inputs2.unsqueeze(1).float()
            labels2 = labels2.float().squeeze()
            #if model == visiom transformer 
            #outputs2 = v(inputs2.permute(0,3,1,2).float())
            #else
            outputs2 = model(inputs2)
            outputs2 = outputs2.squeeze()
            # sum labels
            outputs2 = torch.sum(outputs2,1)

            loss2 = criterion(outputs2, labels2)
            test_loss += loss2.item()

            corr = spearmanr(outputs2.cpu().detach(),labels2.cpu().detach())[0]
            pcc += np.arctanh(corr1)
 
    train_loss = train_loss/len(train_loader)
    test_loss = test_loss/len(valid_loader)
    pcc = pcc/len(valid_loader)
    tr_loss.append(train_loss)
    te_loss.append(test_loss)



    print('epoch ',epoch, 'training loss ', train_loss, 'testing loss ', test_loss)
    print('epoch ',epoch, 'testing correlation ', pcc1)

print('Finished Training/Testing...')
print('[INFO] ellapsed time: ', time.time()-start)

### Results: Model performance 

In [ ]:
# model predictions
out = model(torch.tensor(X_test).unsqueeze(1).float())
#out = model(torch.tensor(imgs_te).permute(0,3,1,2).float()) #if using vision transformer
out_all = out.squeeze().detach().numpy()


# model performance
corr1 = float(spearmanr(pred,y_test1)[0])
corr2 = float(pearsonr(pred,y_test1)[0])
mse = mean_squared_error(pred,y_test1)
mae = mean_absolute_error(pred,y_test1)
r_square = r2_score(y_test1, pred)
res = explained_variance_score(y_test1,pred)

# print output
print('------------------')
print("|Model performance |")
print('--------------------')
print(f"MSE: {round(mse,4)}")
print('-------------------')
print(f"MAE: {round(mae,4)}")
print('--------------------')
print(f"res: {round(res,4)}")
print('------------------------------')
print(f"R_square: {round(r_square,4)}")
print('--------------------------------')
print(f"Spearman corr: {round(corr1,4)}")
print('-------------------------------')
print(f"Pearson corr: {round(corr2,4)}")

### Results: Visualization

In [ ]:
#############################################
# loss and correlation variations over epochs
#############################################

print('-------------------------------------------')
print("loss and correlation variations over epochs")
print('-------------------------------------------')

idx = random.sample(range(1, len(pred)), 20)
plt.plot(tr_loss,color='r',label='train_loss')
plt.plot(te_loss,color='b',label ='valid_loss')
plt.legend()
plt.gca().set_title('loss variations over epochs')

plt.show()
############################
# true vs. predictions plots
############################

print('--------------------------')
print("true vs. predictions plots")
print('--------------------------')

plt.plot(y_test1[idx],color='r',label ='true labels')
plt.plot(pred[idx],color='b',label ='prediction')
plt.legend()
plt.gca().set_title('true scores vs. predicted scores')
plt.show()

##################################
# Features and labels distribution
##################################

''' 
print('---------------------------------------------')
print("Features and labels distribution by threshold")
print('---------------------------------------------')

th = 0.6

random_feat = random_x        
idx1 = np.argwhere(y_test1>th)
idx2 = np.argwhere(y_test1<=th)

feat1 = feat[idx1,:].squeeze(1)
feat2 = feat[idx2,:].squeeze(1)

yt1 = y_test1[idx1]
yt2 = y_test1[idx2]


plt.subplot(1,2,1)
sns.distplot(feat1,label='> th')
sns.distplot(feat2,label='< th')
plt.gca().set_title('features distribution')
plt.legend()

plt.show()
plt.subplot(1,2,1)
plt.hist(yt1)
plt.gca().set_title('labels distribution (score>str(th))')

plt.subplot(1,2,2)
plt.hist(yt2)
plt.gca().set_title('labels distribution (scorestr(th))')

plt.show()
'''
